In [28]:
import os,sys
import cv2

from tqdm import tqdm
import shutil




def get_subfiles(path, length=None):
    out_ls = []

    def _func(x, ls=[]):
        for fn in os.listdir(x):
            full_path = os.path.join(x, fn)
            if os.path.isdir(full_path):
                _func(full_path, ls)
            else:
                ls.append(full_path)
            if length != None and len(out_ls) > length:
                return
        return
    _func(path, out_ls)
    # out_ls.sort()
    return out_ls



In [29]:
from multiprocessing import Process,Pool

src_ffhq_dir = '/media/112new_sde/CommonDatasets/FFHQ/images1024x1024'
dst_dir = '/media/112new_sde/CommonDatasets/dctnet_train/face_photo'

tar_sz=256
fn_s = get_subfiles(src_ffhq_dir)

src_prefix_len = len(src_ffhq_dir)+1


def save_image(src_fn):
    sub_fn = src_fn[src_prefix_len:]
    dst_fn = os.path.join(dst_dir, sub_fn)
    dst_dir_ = os.path.dirname(dst_fn)
    if not os.path.exists(dst_dir_):
        os.makedirs(dst_dir_,exist_ok=True)
    
    srcimg = cv2.imread(src_fn)
    if isinstance(srcimg,type(None)):
        return False
    rsz_img = cv2.resize(srcimg, (tar_sz,tar_sz),interpolation=cv2.INTER_LANCZOS4)
    return cv2.imwrite(dst_fn, rsz_img)

with Pool(48) as p:
    res = list(tqdm(p.imap(save_image, fn_s),total=len(fn_s)))



100%|██████████| 70037/70037 [01:32<00:00, 755.22it/s]


In [33]:
from functools import partial
from multiprocessing import Process,Pool
def mv_file_dummy(fn_pair):
    fn, out_fn = fn_pair
    shutil.move(fn, out_fn)
    return
photo_dir = '/media/112new_sde/CommonDatasets/dctnet_train/face_photo'
style_dir = '/media/112new_sde/CommonDatasets/dctnet_train/face_cartoon'
photo_fns = get_subfiles(photo_dir)
photo_sort_fns = sorted(photo_fns)
photo_fn_pairs = [
    (fn, os.path.join(photo_dir, os.path.join(photo_dir,f'{i:0>6d}.png'))) for i,fn in enumerate(photo_sort_fns)
]
style_fns = get_subfiles(style_dir)
style_sort_fns = sorted(style_fns)
style_fn_pairs = [
    (fn, os.path.join(style_dir, os.path.join(style_dir,f'{i:0>6d}.png'))) for i,fn in enumerate(style_sort_fns)
]

In [34]:
# photo_fn_pairs[0],style_fn_pairs[0]
# photo_fn_pairs[0]
style_fn_pairs[0]

('/media/112new_sde/CommonDatasets/dctnet_train/face_cartoon/raw_style_faces/Adele_0_0.png',
 '/media/112new_sde/CommonDatasets/dctnet_train/face_cartoon/000000.png')

In [35]:
# with Pool(48) as p:
#     res = list(tqdm(p.imap(mv_file_dummy, photo_fn_pairs),total=len(photo_fn_pairs)))
with Pool(48) as p:
    res = list(tqdm(p.imap(mv_file_dummy, style_fn_pairs),total=len(style_fn_pairs)))

100%|██████████| 44123/44123 [00:12<00:00, 3654.82it/s]


In [ ]:
img_fns =[x for x in fn_s if x.endswith('.png') or x.endswith('.jpg')]
print(len(img_fns))
img_fns[0][len(src_ffhq_dir)+1:]